In [17]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame
#from numba import jit

In [18]:
def generate_heston_paths(S, T, r, kappa, theta, v_0, rho, xi, 
                          steps, Npaths, return_vol=False):
    dt = T/steps
    size = (Npaths, steps)
    prices = np.zeros(size)
    sigs = np.zeros(size)
    S_t = S
    v_t = v_0
    for t in range(steps):
        WT = np.random.multivariate_normal(np.array([0,0]), 
                                           cov = np.array([[1,rho],
                                                          [rho,1]]), 
                                           size=Npaths) * np.sqrt(dt) 
        
        S_t = S_t*(np.exp( (r- 0.5*v_t)*dt+ np.sqrt(v_t) *WT[:,0] ) ) 
        v_t = np.abs(v_t + kappa*(theta-v_t)*dt + xi*np.sqrt(v_t)*WT[:,1])
        prices[:, t] = S_t
        sigs[:, t] = v_t
    
    
    return prices

In [19]:
def heston_call(S, K, T, r, kappa, theta, v_0, rho, xi):
    prices_pos = generate_heston_paths(S, T, r, kappa, theta,
                                    v_0, rho, xi, steps=2000, Npaths=20000,
                                    return_vol=False)[:,-1]
    return np.mean(np.maximum(K-prices_pos, 0))*np.exp(-r*T)

In [20]:
np.maximum(-1,0)

0

In [21]:
S_v = np.linspace(10, 500, num = 1000)
T_v = np.linspace(1/12, 3, num= 1000)
r_v = np.linspace(0.01, 0.1, num=100)
v_0_v = np.linspace(0.05, 0.9, num=100)
theta_v = np.linspace(0.01, 0.8, num = 100)
kappa_v = np.linspace(0, 10, num = 100)
rho_v = np.linspace(-0.99, 0.99, num = 1000)
# xi < 2 * theta * kappa
l = [S_v, T_v, r_v, v_0_v, theta_v, kappa_v, rho_v]

In [26]:
param_List = []
for i in range(10000):
    params = []
    for p in l:
        params.append(np.random.choice(p))
    param_List.append(params)

In [27]:
np.random.seed(10086)
for params in param_List:
    S = params[0]
    T = params[1]
    t = np.random.choice(np.linspace(1/12, T, num= 1000))
    q = np.random.choice(np.linspace(-0.2, 0.2, num= 1000))
    K = np.exp(q * t) * S
    params.insert(1, K)
    theta = params[-3]
    kappa = params[-2]
    xi = np.random.choice(np.linspace(0, np.sqrt(2 * theta * kappa), num= 1000))
    params.append(xi)

In [28]:
param_List[2]
# S, K, T, r, v_0, theta, kappa, rho, xi

[104.17417417417417,
 96.21876252042333,
 2.964964964964965,
 0.034545454545454546,
 0.17878787878787877,
 0.16959595959595963,
 1.1111111111111112,
 -0.9761261261261261,
 0.38960566544416725]

In [29]:
for params in tqdm(param_List):
    S = params[0]
    K = params[1]
    T = params[2]
    r = params[3]
    v_0 = params[4]
    theta = params[5]
    kappa = params[6]
    rho = params[7]
    xi = params[8]
    call_price = heston_call(S, K, T, r, kappa, theta, v_0, rho, xi)
    params.append(call_price)

100%|███████████████████████████████████| 10000/10000 [6:22:49<00:00,  2.30s/it]


In [30]:
import pandas as pd
df = pd.DataFrame(param_List)

In [31]:
df.columns = ["S", "K", "T", "r", "v_0", "theta", "kappa", "rho", "xi", "call_price"]

In [32]:
df[df["call_price"] < 0]

,S,K,T,r,v_0,theta,kappa,rho,xi,call_price


In [33]:
heston_call(446.536537, 470.380893, 1.37671, 0.021818, 8.080808, 0.432929, 0.771212, 0.9, 0.908196)

146.78932494853728

In [34]:
np.max([455.04167197953547 - 470.380893, 0])

0.0

In [35]:
df.to_csv("HestonSimulation2.csv")